In [1]:
import numpy as np
import sys
from sklearn.metrics import confusion_matrix, classification_report
from pointcloud import PointCloud
import pandas as pd
import matplotlib.pyplot as plt

from train import split_data, train_svm_linear, train_svm_rbf, train_svm_poly

In [ ]:
labels = ["building", "car", "fence", "pole", "tree"]
labels_dataset = []
features_dataset = []
for hundred in range(5):
    class_name = labels[hundred]
    print(f"Processing {class_name}")
    for i in range(100): #todo 100
        print(f"Processing {i}")
        point_cloud = PointCloud.from_file(f"data/pointclouds/{hundred}{i:02d}.xyz")
        features = point_cloud.compute_feature_matrix()
        labels_dataset.append(class_name)
        features_dataset.append(features)

train_features, test_features, train_labels, test_labels = split_data(features_dataset, labels_dataset)

Processing building
Processing 0
Processing 1
Processing 2
Processing 3
Processing 4
Processing 5
Processing 6
Processing 7
Processing 8
Processing 9
Processing 10
Processing 11
Processing 12
Processing 13
Processing 14
Processing 15
Processing 16
Processing 17
Processing 18
Processing 19
Processing 20
Processing 21
Processing 22
Processing 23
Processing 24
Processing 25
Processing 26
Processing 27
Processing 28
Processing 29
Processing 30


In [6]:
svm_linear = train_svm_linear(train_features, train_labels)
svm_poly = train_svm_poly(train_features, train_labels)
svm_rbf = train_svm_rbf(train_features, train_labels)

In [7]:
y_pred_linear = svm_linear.predict(test_features)
print("Linear Kernel:")
print(confusion_matrix(test_labels, y_pred_linear))
print(classification_report(test_labels, y_pred_linear))

# Polynomial kernel
y_pred_poly = svm_poly.predict(test_features)
print("Polynomial Kernel:")
print(confusion_matrix(test_labels, y_pred_poly))
print(classification_report(test_labels, y_pred_poly))

y_pred_rbf = svm_rbf.predict(test_features)
print("RBF Kernel:")
print(confusion_matrix(test_labels, y_pred_rbf))
print(classification_report(test_labels, y_pred_rbf))

Linear Kernel:
[[4 0 1 0 0]
 [0 5 0 0 0]
 [2 0 1 0 0]
 [0 0 0 4 0]
 [1 0 0 1 1]]
              precision    recall  f1-score   support

    building       0.57      0.80      0.67         5
         car       1.00      1.00      1.00         5
       fence       0.50      0.33      0.40         3
        pole       0.80      1.00      0.89         4
        tree       1.00      0.33      0.50         3

    accuracy                           0.75        20
   macro avg       0.77      0.69      0.69        20
weighted avg       0.78      0.75      0.73        20

Polynomial Kernel:
[[2 0 3 0 0]
 [0 0 5 0 0]
 [0 0 3 0 0]
 [0 0 4 0 0]
 [0 0 3 0 0]]
              precision    recall  f1-score   support

    building       1.00      0.40      0.57         5
         car       0.00      0.00      0.00         5
       fence       0.17      1.00      0.29         3
        pole       0.00      0.00      0.00         4
        tree       0.00      0.00      0.00         3

    accuracy       

/opt/homebrew/Caskroom/miniforge/base/envs/machine_learning/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/machine_learning/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/machine_learning/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete

In [8]:
from sklearn.svm import SVC

svm_rbf_tuned = SVC(kernel='rbf', C=10, gamma=0.1)
svm_rbf_tuned.fit(train_features, train_labels)
y_pred_rbf_tuned = svm_rbf_tuned.predict(test_features)
print("Tuned RBF Kernel:")
print(confusion_matrix(test_labels, y_pred_rbf_tuned))
print(classification_report(test_labels, y_pred_rbf_tuned))

Tuned RBF Kernel:
[[1 0 0 0 4]
 [0 2 0 0 3]
 [0 0 0 0 3]
 [0 0 1 0 3]
 [0 0 0 0 3]]
              precision    recall  f1-score   support

    building       1.00      0.20      0.33         5
         car       1.00      0.40      0.57         5
       fence       0.00      0.00      0.00         3
        pole       0.00      0.00      0.00         4
        tree       0.19      1.00      0.32         3

    accuracy                           0.30        20
   macro avg       0.44      0.32      0.24        20
weighted avg       0.53      0.30      0.27        20



/opt/homebrew/Caskroom/miniforge/base/envs/machine_learning/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/machine_learning/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniforge/base/envs/machine_learning/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` paramete